**AI-Teaching Assistant - Project**

In [1]:
import fitz  
doc = fitz.open("Excerpts - Arendt - The Human Condition.pdf")
text = ""
for page in doc:
    text += page.get_text()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents([text])

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = [doc.page_content for doc in docs]

doc_embeddings = embeddings.embed_documents(texts)

C:\Users\HP\AppData\Local\Temp\ipykernel_331868\4184729053.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [4]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [5]:
retriever = db.as_retriever(search_type="similarity", k=3)
relevant_docs = retriever.get_relevant_documents("What is labor for Arendt?")

C:\Users\HP\AppData\Local\Temp\ipykernel_331868\3270450041.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents("What is labor for Arendt?")


In [20]:
from langchain.chains import RetrievalQA
import os
from huggingface_hub import InferenceClient

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
query = "What is labor for Arendt?"

context = "\n\n".join(doc.page_content for doc in relevant_docs)
final_prompt = f"""Answer the question using the context below. If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{query}
"""


client = InferenceClient(
    provider="hf-inference",
    model= "tiiuae/falcon-7b-instruct",
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

response = client.text_generation(
    prompt=final_prompt,
    max_new_tokens=200,
    temperature=0.3,
)

print(response)

1 
 
Excerpts – The Human Condition - Hannah Arendt (1958) 
 
The Fate of Our Times for Arendt 
 
Closer at hand and perhaps equally decisive is another no less threatening event. This is the 
advent of automation, which in a few decades probably will empty the factories and liberate 
mankind from its oldest and most natural burden, the burden of laboring and the bondage to 
necessity…The modern age has carried with it a theoretical glorification of labor and has

through the interrelated faculties of action and speech, which produce meaningful stories as 
naturally as fabrication produces use objects. 
 
Arendt on how labor (not work or action) are the dominant activity in which human-beings 
participate in the modern world 
 
Among the outstanding characteristics of the modern age from its beginning to our own time we 
find the typical attitudes of homo faber: his instrumentalization of the world, his confidence in

resulted in a factual transformation of the whole of society into a 